# Libraries

In [1]:
from rasa.core.interpreter import RasaNLUInterpreter
from rasa.shared.nlu.training_data.message import Message
from rasa.shared.core.domain import Domain
from rasa.core.featurizers.single_state_featurizer import SingleStateFeaturizer
from rasa.core.agent import load_agent
from rasa.core.channels.channel import UserMessage
from rasa.shared.core.events import SlotSet, Restarted, UserUttered, BotUttered, ActionExecuted
from rasa import model
import numpy as np
import pandas as pd

2021-09-26 12:49:36.219201: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-09-26 12:49:36.219258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Setup

In [2]:
%cd ..

/workspaces/rasa_tracker_feat


In [3]:
model_dir = './models'

### Interpreter

Create an interpreter to parse a user message:

In [4]:
m = model.get_model(model_dir)
_, nlu_model = model.get_model_subdirectories(m)

interpreter = RasaNLUInterpreter(model_directory=nlu_model)

2021-09-26 12:49:38.411292: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-26 12:49:38.411339: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-26 12:49:38.411366: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (32fc601b2629): /proc/driver/nvidia/version does not exist
2021-09-26 12:49:38.411531: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-26 12:49:38.417610: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 229996500

Get the intent mapping from the interpreter's featurizer:

In [5]:
components = interpreter.interpreter.pipeline


featurizers = [c for c in components if 'CountVectorsFeaturizer' in c.unique_name]

has_intent_vectorizer = lambda x: x.vectorizers.get('intent') is not None

featurizer = [f for f in featurizers if has_intent_vectorizer(f)]

assert len(featurizer) == 1

featurizer = featurizer[0]
intent_to_idx_map = featurizer.vectorizers['intent'].vocabulary
intent_to_idx_map

{'inform': 5,
 'greet': 4,
 'goodbye': 3,
 'affirm': 0,
 'deny': 2,
 'mood_great': 6,
 'mood_unhappy': 7,
 'bot_challenge': 1}

Invert the map:

In [6]:
idx_to_intent_map = {v:k for k,v in intent_to_idx_map.items()}
idx_to_intent_map

{5: 'inform',
 4: 'greet',
 3: 'goodbye',
 0: 'affirm',
 2: 'deny',
 6: 'mood_great',
 7: 'mood_unhappy',
 1: 'bot_challenge'}

### State Featurizer

Load the model's domain:

In [7]:
domain = Domain.from_file('./domain.yml')

Create a state featurizer:

In [8]:
ssf = SingleStateFeaturizer()
ssf.prepare_for_training(domain=domain, interpreter=interpreter)

### Agent

Create an agent to create a message processor:

In [9]:
agent = await load_agent(model_path=model_dir, interpreter=interpreter)
message_processor = agent.create_processor()

Get the featurizer for TED Policy too:

In [10]:
ted_featurizer = [p for p in agent.policy_ensemble.policies if p.__class__.__name__ == 'TEDPolicy'][0].featurizer

# Utility Functions

In [11]:
def create_user_message(text, sender_id="test_id"):
    return UserMessage(text=text, sender_id=sender_id)

In [12]:
async def get_tracker(message_processer, user_message, save_tracker=False):
    return await message_processor.log_message(user_message, should_save_tracker = save_tracker)

In [13]:
async def create_prediction_states_and_features(user_message):
    tracker = await get_tracker(message_processor, user_message)
    states =  ted_featurizer.prediction_states([tracker], domain)[0]
    features = ted_featurizer.create_state_features([tracker], domain, interpreter)[0]
    
    return states, features

In [14]:
def extract_features(features, feature_type):
    
    if feature_type == 'intent':
        f = features[feature_type]
        assert len(f) == 1
        
        return f[0].features.todense()
    
    elif feature_type == 'entities':
        f = features[feature_type]
        assert len(f) == 1
        f = f[0].features
        assert f.shape[0] == 1
        
        return np.array(f.todense()).squeeze()
    
    elif feature_type == 'slots':
        f = features[0]
        assert len(f[feature_type]) == 1
        f = f[feature_type][0]
        
        return np.array(f.features.todense()).squeeze()

In [15]:
def get_slot_prediction_states_and_features(tracker, slot_event):
    tracker.update(Restarted(), domain)
    tracker.update(slot_event, domain)
    
    states = ted_featurizer.prediction_states([tracker], domain)[0]
    features = ted_featurizer.create_state_features([t], domain, interpreter)[0]
    
    return states, features

# Featurization

## Intent

Define a user messsage:

In [16]:
message = "I am feeling happy"
user_message = UserMessage(text=message, sender_id="test_id")

Parse the user message:

In [17]:
await message_processor.parse_message(user_message)

{'text': 'I am feeling happy',
 'intent': {'id': 5881328340418250069,
  'name': 'mood_great',
  'confidence': 0.9999083280563354},
 'entities': [],
 'intent_ranking': [{'id': 5881328340418250069,
   'name': 'mood_great',
   'confidence': 0.9999083280563354},
  {'id': -6694443392625421003,
   'name': 'bot_challenge',
   'confidence': 3.876308983308263e-05},
  {'id': -7509306822527489582,
   'name': 'mood_unhappy',
   'confidence': 3.4971741115441546e-05},
  {'id': 4390007749353229196,
   'name': 'deny',
   'confidence': 6.9123402681725565e-06},
  {'id': 5131837534165408679,
   'name': 'greet',
   'confidence': 5.682764822267927e-06},
  {'id': 1335942896814238086,
   'name': 'affirm',
   'confidence': 3.985840521636419e-06},
  {'id': -4713551111634206545,
   'name': 'inform',
   'confidence': 1.2289583537494764e-06},
  {'id': -7784688068645847004,
   'name': 'goodbye',
   'confidence': 1.1073764483171544e-07}],
 'response_selector': {'all_retrieval_intents': [],
  'default': {'response':

Get the state and features of the user message:

In [18]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'mood_great'},
 'prev_action': {'action_name': 'action_listen'}}

Manually compute the features given the states:

In [19]:
intent_features_manual = interpreter.featurize_message(Message(data=states['user']))

intent_features_manual = intent_features_manual.features[0].features.todense()
intent_features_manual

matrix([[0, 0, 0, 0, 0, 0, 1, 0]])

Validate the manual computation:

In [20]:
assert np.array_equal(intent_features_manual, extract_features(features, 'intent'))

Double check using the text:

In [21]:
idx = intent_features_manual.argmax()
idx_to_intent_map[idx]

'mood_great'

## Entities

This is how the entities are mapped:

In [22]:
entity_to_idx = ssf._default_feature_states['entities']
idx_to_entity = {v:k for k,v in entity_to_idx.items()}

entity_to_idx

{'account': 0,
 'city': 1,
 'topping': 2,
 'size': 3,
 'city#departure': 4,
 'city#destination': 5,
 'topping#1': 6,
 'topping#2': 7,
 'size#1': 8,
 'size#2': 9}

### Entities With Roles

Define a message:

In [23]:
message = "I want to travel from berlin to san francisco"
user_message = UserMessage(text=message, sender_id="test_id")

Process the message:

In [24]:
await message_processor.parse_message(user_message)

{'text': 'I want to travel from berlin to san francisco',
 'intent': {'id': -4713551111634206545,
  'name': 'inform',
  'confidence': 0.9999123811721802},
 'entities': [{'entity': 'city',
   'start': 22,
   'end': 28,
   'confidence_entity': 0.9618536829948425,
   'value': 'berlin',
   'extractor': 'DIETClassifier'},
  {'entity': 'city',
   'start': 32,
   'end': 45,
   'confidence_entity': 0.915094256401062,
   'role': 'destination',
   'confidence_role': 0.7696428894996643,
   'value': 'san francisco',
   'extractor': 'DIETClassifier'}],
 'intent_ranking': [{'id': -4713551111634206545,
   'name': 'inform',
   'confidence': 0.9999123811721802},
  {'id': -6694443392625421003,
   'name': 'bot_challenge',
   'confidence': 3.93703012377955e-05},
  {'id': -7784688068645847004,
   'name': 'goodbye',
   'confidence': 1.9640792743302882e-05},
  {'id': 1335942896814238086,
   'name': 'affirm',
   'confidence': 1.5616844393662177e-05},
  {'id': 4390007749353229196,
   'name': 'deny',
   'confid

Create the states and features:

In [25]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'inform', 'entities': ('city', 'city#destination')},
 'prev_action': {'action_name': 'action_listen'}}

Compute the features:

In [26]:
sub_state = states['user']

entity_features_manual = ssf._create_features(sub_state, 'entities')[0].features.flatten()
entity_features_manual

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

Validate the manual computation:

In [27]:
assert np.array_equal(entity_features_manual, extract_features(features, 'entities'))

View the results in text:

In [28]:
[idx_to_entity[i.item()] for i in entity_features_manual.nonzero()[0]]

['city', 'city#destination']

### Entities With Groups

In [29]:
message = "Give me a small pizza with mushrooms and a large pepperoni"
user_message = UserMessage(text=message, sender_id="test_user")

In [30]:
await message_processor.parse_message(user_message)

{'text': 'Give me a small pizza with mushrooms and a large pepperoni',
 'intent': {'id': -4713551111634206545,
  'name': 'inform',
  'confidence': 0.9999113082885742},
 'entities': [{'entity': 'size',
   'start': 10,
   'end': 15,
   'confidence_entity': 0.9797298908233643,
   'group': '1',
   'confidence_group': 0.9713470935821533,
   'value': 'small',
   'extractor': 'DIETClassifier'},
  {'entity': 'topping',
   'start': 27,
   'end': 36,
   'confidence_entity': 0.9666152596473694,
   'group': '1',
   'confidence_group': 0.9583228230476379,
   'value': 'mushrooms',
   'extractor': 'DIETClassifier'},
  {'entity': 'size',
   'start': 43,
   'end': 48,
   'confidence_entity': 0.971481442451477,
   'group': '2',
   'confidence_group': 0.9579463601112366,
   'value': 'large',
   'extractor': 'DIETClassifier'},
  {'entity': 'topping',
   'start': 49,
   'end': 58,
   'confidence_entity': 0.9747665524482727,
   'group': '2',
   'confidence_group': 0.9859060049057007,
   'value': 'pepperoni'

In [31]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'inform',
  'entities': ('size',
   'size#1',
   'size#2',
   'topping',
   'topping#1',
   'topping#2')},
 'prev_action': {'action_name': 'action_listen'}}

In [32]:
sub_state = states['user']

entity_features_manual = ssf._create_features(sub_state, 'entities')[0].features.flatten()
entity_features_manual

array([0., 0., 1., 1., 0., 0., 1., 1., 1., 1.], dtype=float32)

In [33]:
assert np.array_equal(entity_features_manual, extract_features(features, 'entities'))

In [34]:
[idx_to_entity[i.item()] for i in entity_features_manual.nonzero()[0]]

['topping', 'size', 'topping#1', 'topping#2', 'size#1', 'size#2']

### Single Entity

In [35]:
message = "I want to open a checking account"
user_message = UserMessage(text=message, sender_id="test_user")

In [36]:
await message_processor.parse_message(user_message)

{'text': 'I want to open a checking account',
 'intent': {'id': -4713551111634206545,
  'name': 'inform',
  'confidence': 0.9998669624328613},
 'entities': [{'entity': 'account',
   'start': 17,
   'end': 33,
   'confidence_entity': 0.9945732951164246,
   'value': 'checking account',
   'extractor': 'DIETClassifier'}],
 'intent_ranking': [{'id': -4713551111634206545,
   'name': 'inform',
   'confidence': 0.9998669624328613},
  {'id': -6694443392625421003,
   'name': 'bot_challenge',
   'confidence': 7.860726327635348e-05},
  {'id': 1335942896814238086,
   'name': 'affirm',
   'confidence': 2.9824941520928405e-05},
  {'id': -7784688068645847004,
   'name': 'goodbye',
   'confidence': 1.7919752281159163e-05},
  {'id': 4390007749353229196,
   'name': 'deny',
   'confidence': 2.527108335925732e-06},
  {'id': -7509306822527489582,
   'name': 'mood_unhappy',
   'confidence': 1.922559249578626e-06},
  {'id': 5881328340418250069,
   'name': 'mood_great',
   'confidence': 1.0976101521009696e-06

In [37]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'inform', 'entities': ('account',)},
 'prev_action': {'action_name': 'action_listen'}}

In [38]:
sub_state = states['user']

entity_features_manual = ssf._create_features(sub_state, 'entities')[0].features.flatten()
entity_features_manual

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [39]:
assert np.array_equal(entity_features_manual, extract_features(features, 'entities'))

In [40]:
[idx_to_entity[i.item()] for i in entity_features_manual.nonzero()[0]]

['account']

### Repeat Entities

In [41]:
message = "I want to transfer money from my checking account to my savings account"
user_message = UserMessage(text=message, sender_id="test_user")

In [42]:
await message_processor.parse_message(user_message)

{'text': 'I want to transfer money from my checking account to my savings account',
 'intent': {'id': -4713551111634206545,
  'name': 'inform',
  'confidence': 0.9999291300773621},
 'entities': [{'entity': 'account',
   'start': 33,
   'end': 49,
   'confidence_entity': 0.9910616874694824,
   'value': 'checking account',
   'extractor': 'DIETClassifier'},
  {'entity': 'account',
   'start': 56,
   'end': 71,
   'confidence_entity': 0.9932772517204285,
   'value': 'savings account',
   'extractor': 'DIETClassifier'}],
 'intent_ranking': [{'id': -4713551111634206545,
   'name': 'inform',
   'confidence': 0.9999291300773621},
  {'id': -6694443392625421003,
   'name': 'bot_challenge',
   'confidence': 3.596628812374547e-05},
  {'id': 1335942896814238086,
   'name': 'affirm',
   'confidence': 1.93811265489785e-05},
  {'id': -7784688068645847004,
   'name': 'goodbye',
   'confidence': 1.0578203728073277e-05},
  {'id': 4390007749353229196,
   'name': 'deny',
   'confidence': 2.022358103204169

In [43]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'inform', 'entities': ('account',)},
 'prev_action': {'action_name': 'action_listen'}}

In [44]:
sub_state = states['user']

entity_features_manual = ssf._create_features(sub_state, 'entities')[0].features.flatten()
entity_features_manual

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [45]:
assert np.array_equal(entity_features_manual, extract_features(features, 'entities'))

In [46]:
[idx_to_entity[i.item()] for i in entity_features_manual.nonzero()[0]]

['account']

### Different Entity Types

In [47]:
message = "I want to open a savings account and travel to Berlin"
user_message = UserMessage(text=message, sender_id="test_user")

In [48]:
await message_processor.parse_message(user_message)

{'text': 'I want to open a savings account and travel to Berlin',
 'intent': {'id': -4713551111634206545,
  'name': 'inform',
  'confidence': 0.9999034404754639},
 'entities': [{'entity': 'account',
   'start': 17,
   'end': 32,
   'confidence_entity': 0.990652322769165,
   'value': 'savings account',
   'extractor': 'DIETClassifier'},
  {'entity': 'city',
   'start': 47,
   'end': 53,
   'confidence_entity': 0.9859669804573059,
   'value': 'Berlin',
   'extractor': 'DIETClassifier'}],
 'intent_ranking': [{'id': -4713551111634206545,
   'name': 'inform',
   'confidence': 0.9999034404754639},
  {'id': -6694443392625421003,
   'name': 'bot_challenge',
   'confidence': 5.4289816034724936e-05},
  {'id': -7784688068645847004,
   'name': 'goodbye',
   'confidence': 1.696464460110292e-05},
  {'id': 1335942896814238086,
   'name': 'affirm',
   'confidence': 1.6590651284786873e-05},
  {'id': 4390007749353229196,
   'name': 'deny',
   'confidence': 4.576474111672724e-06},
  {'id': -7509306822527

In [49]:
states, features = await create_prediction_states_and_features(user_message)
states = states[-1]
features = features[-1]

states

{'user': {'intent': 'inform', 'entities': ('account', 'city')},
 'prev_action': {'action_name': 'action_listen'}}

In [50]:
sub_state = states['user']

entity_features_manual = ssf._create_features(sub_state, 'entities')[0].features.flatten()
entity_features_manual

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [51]:
assert np.array_equal(entity_features_manual, extract_features(features, 'entities'))

In [52]:
[idx_to_entity[i.item()] for i in entity_features_manual.nonzero()[0]]

['account', 'city']

## Slots

This is how the slots are mapped to indices:

In [53]:
s = ssf._default_feature_states['slots']
s

{'cuisine_0': 0,
 'is_authenticated_0': 1,
 'is_authenticated_1': 2,
 'risk_level_0': 3,
 'risk_level_1': 4,
 'risk_level_2': 5,
 'risk_level_3': 6,
 'temperature_0': 7,
 'temperature_1': 8,
 'shopping_items_0': 9}

Let's create a dataframe to store the values:

In [54]:
df_slot_features = pd.DataFrame(s, index = [0]).transpose().rename(columns = {0:"value"})
df_slot_features['value'] = pd.NA
df_slot_features

,value
cuisine_0,<NA>
is_authenticated_0,<NA>
is_authenticated_1,<NA>
risk_level_0,<NA>
risk_level_1,<NA>
risk_level_2,<NA>
risk_level_3,<NA>
temperature_0,<NA>
temperature_1,<NA>
shopping_items_0,<NA>


A tracker to store the slot events:

In [55]:
t = await get_tracker(message_processor, user_message)

### Boolean Slots

If the value is False: 

In [56]:
s = SlotSet("is_authenticated", False)

In [57]:
states, features = get_slot_prediction_states_and_features(t, s)
states

[{'slots': {'is_authenticated': (1.0, 0.0)}}]

In [58]:
slot_features = extract_features(features, 'slots')
slot_features

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [59]:
df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,1
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


If the value is True:

In [60]:
s = SlotSet("is_authenticated", True)

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,1
is_authenticated_1,1
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


### Categorical Slots

If the value is one of the defined levels:

In [61]:
s = SlotSet("risk_level", "low")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,1
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


In [62]:
s = SlotSet("risk_level", "medium")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,1
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


In [63]:
s = SlotSet("risk_level", "high")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,1
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


If the value is not one of the defined levels:

In [64]:
s = SlotSet("risk_level", "super high")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,1
temperature_0,0
temperature_1,0
shopping_items_0,0


### Text Slots

Any value will make the slot a 1:

In [65]:
s = SlotSet("cuisine", "foo")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,1
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


In [66]:
s = SlotSet("cuisine", "bar")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,1
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,0


### Float Slots

If value is within the bounds:

In [67]:
s = SlotSet("temperature", 0)

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,1
temperature_1,0
shopping_items_0,0


If value is less than the lower bound:

In [68]:
s = SlotSet("temperature", -200)

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,1
temperature_1,0
shopping_items_0,0


If value is more than the upper bound:

In [69]:
s = SlotSet("temperature", 200)

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,1
temperature_1,1
shopping_items_0,0


### List Slots

A non empty list:

In [70]:
s = SlotSet("shopping_items", ["eggs", "milk"])

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,1


But you don't have to pass a list:

In [71]:
s = SlotSet("shopping_items", 200)

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,1


In [72]:
s = SlotSet("shopping_items", "eggs")

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,1


A list of None will get featurized too:

In [73]:
s = SlotSet("shopping_items", [None])

states, features = get_slot_prediction_states_and_features(t, s)

slot_features = extract_features(features, 'slots')

df_slot_features['value'] = slot_features.astype('int')
df_slot_features

,value
cuisine_0,0
is_authenticated_0,0
is_authenticated_1,0
risk_level_0,0
risk_level_1,0
risk_level_2,0
risk_level_3,0
temperature_0,0
temperature_1,0
shopping_items_0,1


But an empty list will not:

In [74]:
import traceback

s = SlotSet("shopping_items", [])

states, features = get_slot_prediction_states_and_features(t, s)

try:
    slot_features = extract_features(features, 'slots')
except KeyError as e:
    print(f"features is {features}")
    traceback.print_exc()

features is [{}]


Traceback (most recent call last):
  File "/tmp/ipykernel_21931/1118870660.py", line 8, in <module>
    slot_features = extract_features(features, 'slots')
  File "/tmp/ipykernel_21931/2479924411.py", line 19, in extract_features
    assert len(f[feature_type]) == 1
KeyError: 'slots'


# Appendix

## Making Up A Tracker

Make a message:

In [75]:
user_message = UserMessage("hello", sender_id="fake_user2")

Start a tracker with that message:

In [76]:
t = await get_tracker(message_processor, user_message)
t.events

deque([ActionExecuted(action: action_session_start, policy: None, confidence: 1.0),
       ActionExecuted(action: action_listen, policy: None, confidence: None),
       <rasa.shared.core.events.UserUttered at 0x7f929876ed90>])

Make up some events following the user's message:

In [77]:
user_message = UserMessage("I want to open a checking account", sender_id="fake_user")
parsed_message = await message_processor.parse_message(user_message)
u = UserUttered(text = parsed_message['text'],
            intent = parsed_message['intent'],
            entities = parsed_message['entities'])


my_events = [ActionExecuted(action_name="utter_greet"), 
             SlotSet("risk_level", "low"), 
             u,
             ActionExecuted(action_name="utter_happy")
            ]

t.update_with_events(my_events, domain)

View the saved events:

In [78]:
t.events

deque([ActionExecuted(action: action_session_start, policy: None, confidence: 1.0),
       ActionExecuted(action: action_listen, policy: None, confidence: None),
       ActionExecuted(action: utter_greet, policy: None, confidence: None),
       ActionExecuted(action: utter_happy, policy: None, confidence: None)])

Convert the events into states for prediction:

In [79]:
states =  ted_featurizer.prediction_states([t], domain)[0]
states 

[{},
 {'user': {'intent': 'greet'},
  'prev_action': {'action_name': 'action_listen'}},
 {'user': {'intent': 'inform', 'entities': ('account',)},
  'slots': {'risk_level': (1.0, 0.0, 0.0, 0.0)},
  'prev_action': {'action_name': 'utter_greet'}},
 {'user': {'intent': 'inform', 'entities': ('account',)},
  'slots': {'risk_level': (1.0, 0.0, 0.0, 0.0)},
  'prev_action': {'action_name': 'utter_happy'}}]

The features of each state:

In [80]:
features = ted_featurizer.create_state_features([t], domain, interpreter)[0]
features

[{},
 {'intent': [<rasa.shared.nlu.training_data.features.Features at 0x7f94021fa790>],
  'action_name': [<rasa.shared.nlu.training_data.features.Features at 0x7f92986f9190>]},
 {'slots': [<rasa.shared.nlu.training_data.features.Features at 0x7f94021fa7c0>],
  'action_name': [<rasa.shared.nlu.training_data.features.Features at 0x7f92987669a0>]},
 {'slots': [<rasa.shared.nlu.training_data.features.Features at 0x7f9298766ac0>],
  'action_name': [<rasa.shared.nlu.training_data.features.Features at 0x7f92993bfac0>]}]